## Classification with TensorFlow Decision Forests

Models include Random forests, gradient boosted trees and carts, and can be used for regression, classification and raking task

Example below used gradient boosted trees model in binary classification of structured data, and covers the following:
- Build a decision forests model by specifying the input
- implement a custom binary target encoder
- Encode the categorical features as embedding, train these embdding in simple nn model

In [1]:
import math
import urllib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_decision_forests as tfdf

### Prepare the data

Dataset will be from the united states census income dataset provided by uc irvine

Example will be an binary classification to determine whether a person makes over 50k a year

In [2]:
BASE_PATH = "https://archive.ics.uci.edu/ml/machine-learning-databases/census-income-mld/census-income"

CSV_HEADER = [
    l.decode("utf-8").split(":")[0].replace(" ", "_")
    for l in urllib.request.urlopen(f"{BASE_PATH}.names")
    if not l.startswith(b"|")
][2:]

CSV_HEADER.append("income_level")

train_data = pd.read_csv(f"{BASE_PATH}.data.gz", header=None, names=CSV_HEADER,)
test_data = pd.read_csv(f"{BASE_PATH}.test.gz", header=None, names=CSV_HEADER,)

### Define dataset metadata

Define the metadata of the dataset tha twill be useful for encoding the input featuers 
